# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-01-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-03-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-02-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-13-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-08-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# what is the meaning of class/ replication factor 
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    {'class':'SimpleStrategy', 'replication_factor':1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try: 
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1="""
CREATE TABLE IF NOT EXISTS artist_song (sessionId int, itemInSession int, artist varchar,\
song_title varchar, length float, PRIMARY KEY (sessionId, itemInSession))
"""
try:
    session.execute(query1)
except Exception as e:
    print(e)
                    

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_song (sessionId, itemInSession, artist, song_title, length)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        try:
    ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
            session.execute(query, (int(line[8]), int(line[3]),line[0],line[9],float(line[5])))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query="SELECT * FROM artist_song"
rows=session.execute(query)
for i in rows:
    print(i.artist,i.song_title,i.length)

Regina Spektor The Calculation (Album Version) 191.08526611328125
Octopus Project All Of The Champs That Ever Lived 250.95791625976562
Tegan And Sara So Jealous 180.06158447265625
Dragonette Okay Dolores 153.39056396484375
Lil Wayne / Eminem Drop The World 229.58975219726562
Soulja Boy Tell'em Let Me Get Em 201.1162872314453
Bodo Wartke Liebeslied (Sprachen: Deutsch_ Englisch_ FranzÃÂ¶sisch_ Italienisch_ Spanisch_ HollÃÂ¤ndisch_ Japanisch_ Russisch_ Griechisch_ Klingonisch_ Hessisch) 645.2763061523438
Evanescence Bring Me To Life 237.11302185058594
Van Halen Good Enough 243.1734161376953
The Academy Is... Paper Chase (Album Version) 209.7628173828125
Dwight Yoakam You're The One 239.3072967529297
The Far East Movement featuring Wiz Khalifa and Bionik Lowridin 265.50811767578125
Amon Amarth The Hero 238.5236358642578
Modest Mouse The Good Times Are Killing Me 259.47381591796875
Bad Company Morning Sun 246.961181640625
Bodyrox Yeah Yeah 490.7881164550781
Alberto Plaza Yo Te Seguire (vi

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query2="""
CREATE TABLE IF NOT EXISTS song_playlist_session (artist varchar,userid int, song_title varchar, firstName varchar, lastName varchar,\
itemInSession int, sessionId int, PRIMARY KEY ((userid,sessionId), itemInSession))
"""
try:
    session.execute(query2)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_playlist_session (userid, sessionId, itemInSession, artist, song_title, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

        
        try:
            session.execute(query, (int(line[10]),int(line[8]), int(line[3]),line[0],line[9],line[1],line[4]))
        except Exception as e:
            print (e)
                    

In [12]:
query = "SELECT artist, song_title, firstName, lastName from song_playlist_session where sessionId= 182 and userid = 10"
try: 
    rows = session.execute(query)
except Exception as e:
    print (e)

for row in rows:
    print (row)

Row(artist='Down To The Bone', song_title="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song_title='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song_title='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [15]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query3="CREATE TABLE IF NOT EXISTS user (userId int, firstName varchar, lastName varchar, song_title varchar, \
PRIMARY KEY(song_title, userid))"

try:
    session.execute(query3)
except Exception as e:
    print(e)

file='event_datafile_new.csv'

with open(file,encoding='utf8') as f:
    
    csvreader=csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query="INSERT INTO user (userid, firstName, lastName, song_title) VALUES (%s, %s, %s, %s)"
        
        try:
            session.execute(query, (int(line[10]),line[1],line[4],line[9]))
        except Exception as e:
            
            print(e)
                    

In [16]:
query="SELECT * FROM user WHERE song_title='All Hands Against His Own' "

try:
    rows=session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [ ]:
# drop table artist_song
query='DROP TABLE artist_song'
try:
    rows=session.execute(query)
except Exception as e:    
    print(e)

# drop table song_playlist_session 
query = "DROP TABLE song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# drop table user 
query = "DROP TABLE  user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()